In [4]:
from generate_attestations import *

num_wallets = 10 
wallet_addresses = generate_wallet_addresses(num_wallets)
attestations = generate_attestations(10,wallet_addresses=wallet_addresses)


In [7]:
from graphing import *
import networkx as nx
import matplotlib.pyplot as plt

G = create_network_graph(wallet_addresses,attestations)
draw_network_graph(G)

{'isHuman': True, 'isBot': True, 'creditworthinessScore': True}